In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
df1=pd.read_csv("reviews_det.csv")
df1 = df1.rename(columns={'id': 'review_id'})
df1=df1.rename(columns={'listing_id': 'id'})
df1.head()
df1.shape

(431830, 6)

In [3]:
df2=pd.read_csv("listing_det.csv")
df2.head()
df2.shape

/opt/apps/rhel8/Anaconda3-2021.05/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(20030, 96)

In [4]:
df=pd.merge(df1, df2, on='id', how='left')
df.head()


,id,review_id,date,reviewer_id,reviewer_name,comments,listing_url,scrape_id,last_scraped,name,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2818,1191,3/30/2009,10952,Lam,Daniel is really cool. The place was nice and ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
1,2818,1771,4/24/2009,12798,Alice,Daniel is the most amazing host! His place is ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
2,2818,1989,5/3/2009,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
3,2818,2797,5/18/2009,14064,Enrique,Very professional operation. Room is very clea...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1
4,2818,3151,5/25/2009,17977,Sherwin,Daniel is highly recommended. He provided all...,https://www.airbnb.com/rooms/2818,2.018120e+13,12/6/2018,Quiet Garden View Room & Super Fast WiFi,...,f,NaN,{Amsterdam},t,f,strict_14_with_grace_period,f,f,1,2.1


In [5]:
df.shape

(431830, 101)

In [5]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /hpc/home/nsb37/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
def get_rating(review):
    # Compute the sentiment score using Vader
    sentiment = sia.polarity_scores(review)['compound']
    
    # Convert the sentiment score to a rating from 5.0 to 0.5
    if sentiment >= 0.9:
        rating = 5.0
    elif sentiment >= 0.4:
        rating = 4.5
    elif sentiment >= 0.1:
        rating = 4.0
    elif sentiment >= -0.3:
        rating = 3.5
    elif sentiment >= -0.8:
        rating = 3.0
    elif sentiment >= -0.95:
        rating = 2.5
    elif sentiment >= -0.99:
        rating = 2.0
    elif sentiment >= -1.0:
        rating = 1.5
    elif sentiment >= -1.5:
        rating = 1.0
    else:
        rating = 0.5
    
    return rating


In [7]:
def get_rating(review):
    # Compute the sentiment score using Vader
    sentiment = sia.polarity_scores(review)['compound']
    
    # Convert the sentiment score to a rating from 5.0 to 0.5
    if sentiment >= 0.9:
        rating = 5.0
    elif sentiment >= 0.4:
        rating = 4.5
    elif sentiment >= 0.1:
        rating = 4.0
    elif sentiment >= -0.3:
        rating = 3.5
    elif sentiment >= -0.8:
        rating = 3.0
    elif sentiment >= -0.95:
        rating = 2.5
    elif sentiment >= -0.99:
        rating = 2.0
    elif sentiment >= -1.0:
        rating = 1.5
    elif sentiment >= -1.5:
        rating = 1.0
    else:
        rating = 0.5
    
    return rating


In [7]:
df['ratings'] = df['comments'].astype(str).apply(get_rating)

In [8]:
unique_values = df['ratings'].value_counts()
print(unique_values)

5.0    230567
4.5    118476
3.5     54685
3.0     10208
4.0      9006
2.5      5960
2.0      2702
1.5       226
Name: ratings, dtype: int64


In [18]:
rating_counts = df['ratings'].value_counts()

# Print the results
print(rating_counts)

5    230567
4    109011
2     57833
3     18471
1     15948
Name: ratings, dtype: int64


In [9]:
df.to_csv('combined_data_fin.csv', index=False)